In [21]:
import pandas as pd
from statsmodels.tsa.seasonal import seasonal_decompose
from matplotlib import pyplot
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import MinMaxScaler
import os

In [22]:
df = pd.read_csv('./data/energia_com_cnpj.csv', sep=',')

<ipython-input-22-93a3eeef811a>:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/energia_com_cnpj.csv', sep=',')


- __DEM_X_01-12__: Os valores de demanda são a demanda ativa máxima efetivamente lida na
unidade para o período de referência correspondente, segregando ponta e fora ponta para
AT. Deve-se informar, caso não seja possível realizar a leitura, os valores faturados. Deve-se
informar, para o caso de unidades sem medição, os valores faturados. Deve-se informar, caso
haja post. Unidade: kWh

-  __ENE_X_01-12__: Os valores de energia são a energia ativa consumida ou gerada efetivamente
lida na unidade para o período de referência correspondente, segregando ponta e fora ponta
para AT. Deve-se informar, caso não haja consumo, o valor zero. Deve-se informar, caso não
seja possível realizar a leitura, os valores faturados. Deve-se informar, para caso de unidades
sem medição, os valores faturados. Deve-se informar, caso haja postos horários segregados,
o somatório da energia destes postos. Unidade: kWh

- __DIC_XX, FIC_XX__: Deve apresentar, respectivamente, os valores mensais de duração (em
horas) e frequência das interrupções individuais apurados, do acessante ou ponto de
iluminação, considerando o período de referência da base de dados (para uma base ordinária
deve coincidir com o valor mensal do ano anterior e para uma base extraordinária os valores
apurados nos últimos 12 meses). Unidade: horas


In [23]:
df_municipios = pd.read_csv('./data/municipios/municipios.csv', sep=';', header=None)
df_municipios.columns = ['municipio', 'nome_municipio']
df_municipios

,municipio,nome_municipio
0,1,GUAJARA-MIRIM
1,2,ALTO ALEGRE DOS PARECIS
2,3,PORTO VELHO
3,4,BURITIS
4,5,JI-PARANA
...,...,...
5566,9989,SERRA ALTA
5567,9991,TUNAPOLIS
5568,9993,GUARINOS
5569,9995,RIO QUENTE


In [24]:
df_ucmt = pd.read_csv('./data/bdgd/ucmt_pj.csv', sep=';', encoding='ISO-8859-1')
df_ucmt = df_ucmt[~df_ucmt['COD_ID'].isin(df['COD_ID'].unique())]

<ipython-input-24-f8a2c9c3c59e>:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ucmt = pd.read_csv('./data/bdgd/ucmt_pj.csv', sep=';', encoding='ISO-8859-1')


In [25]:
df_ucmt

,COD_ID,DIST,PN_CON,PAC,CTMT,UNI_TR_AT,SUB,CONJ,MUN,CEG_GD,...,FIC_08,FIC_09,FIC_10,FIC_11,FIC_12,SEMRED,DESCR,DATA_BASE,POINT_X,POINT_Y
0,41799127,385,2109274265,LEM142109274265,LEM01_AL014,LEM01_ETR01,LEM01,13529,3526704,NaN,...,0,0,0,0,0,0,NaN,31DEC2022:00:00:00.0000000,-47.379923,-22.175056
1,34432850,385,53136023,MOM0653136023,MOM01_AL006,MOM01_ETR01,MOM01,13540,3530805,NaN,...,0,0,0,0,0,0,NaN,31DEC2022:00:00:00.0000000,-46.982702,-22.438672
2,18447660,385,2109250924,SRP062109250924,SRP01_AL006,SRP01_ETR01,SRP01,13585,3547502,NaN,...,0,1,1,1,1,0,NaN,31DEC2022:00:00:00.0000000,-47.507915,-21.712679
3,28780523,385,24038265,JAC1224038265,CAJ01_AL021,CAJ01_ETR03,CAJ01,13470,3524600,NaN,...,0,0,0,0,2,0,NaN,31DEC2022:00:00:00.0000000,-47.996874,-24.691521
4,70198922551,7019,1004701734,337M-173055,MC-03,MCTF4-02,5003222,12585,1302504,NaN,...,0,0,0,0,0,0,1019106047,31DEC2022:00:00:00.0000000,-60.622424,-3.301761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262410,17205,2763,104044.0,1040441,203_CER01_2,2_CER01_4,CER01,15446,4309001,NaN,...,0,0,1,4,1,0,NaN,31DEC2022:00:00:00.0000000,-54.075966,-28.090008
262411,17945,2763,117774.0,1177741,203_CER02_2,2_CER02_4,CER02,15446,4305405,NaN,...,1,0,3,2,1,0,NaN,31DEC2022:00:00:00.0000000,-53.868868,-28.061166
262413,18715,2763,121548.0,1215481,204_CER02_2,2_CER02_4,CER02,15446,4305405,NaN,...,0,6,1,0,0,0,NaN,31DEC2022:00:00:00.0000000,-53.913939,-27.923913
262414,17474,2763,104951.0,1049511,203_CER01_2,2_CER01_4,CER01,15446,4310405,NaN,...,0,3,1,4,0,0,NaN,31DEC2022:00:00:00.0000000,-54.093267,-28.006611


In [26]:
df = df.merge(df_municipios, on='municipio', how='left')
df = pd.concat([df, df_ucmt], ignore_index=True)
df.drop_duplicates(subset=['COD_ID'], inplace=True)
#droping rows that has zero or negatives values in ENE_ columns
ene_cols = ['ENE_09', 'ENE_10', 'ENE_11', 'ENE_12']#[col for col in df.columns if 'ENE_' in col]
ene_cols_full = [col for col in df.columns if 'ENE_' in col]
df = df[(df[ene_cols_full] > 0).all(axis=1)].reset_index(drop=True)

df['consumo_minimo_ultimo_ano'] = df[ene_cols_full].min(axis=1)
df['consumo_maximo_ultimo_ano'] = df[ene_cols_full].max(axis=1)
df['consumo_medio_ultimo_ano'] = df[ene_cols_full].mean(axis=1)
df['consumo_mediano_ultimo_ano'] = df[ene_cols_full].median(axis=1)
df['desvio_padrao_consumo_ultimo_ano'] = df[ene_cols_full].std(axis=1)
df['consumo_final_menos_inicial'] = df['ENE_12'] - df['ENE_09']
#vairacao percentual do consumo de energia de janeiro a dezembro em porcentagem
df['variacao_percentual_consumo'] = 100*(df['ENE_12'] - df['ENE_09']) / df['ENE_09']
len(df)

200557

In [27]:
df_ene_full = df[ene_cols_full].T.rolling(window=4).mean().dropna()
df_ene_full.columns = df['COD_ID']
#transforming the row to list
df_ene_full = df_ene_full.T
ene_mov_cols = [col + '_MA' for col in df_ene_full.columns if 'ENE_' in col]
df_ene_full.columns = ene_mov_cols
df_ene_full.reset_index(inplace=True)
df_ene_full

,COD_ID,ENE_04_MA,ENE_05_MA,ENE_06_MA,ENE_07_MA,ENE_08_MA,ENE_09_MA,ENE_10_MA,ENE_11_MA,ENE_12_MA
0,42836184,36013.250000,40023.250000,41690.000000,40304.500000,38380.000000,41512.500000,41576.250000,41008.500000,39941.000000
1,4000859701,462.250000,457.896750,460.896750,402.146750,386.646750,378.250000,390.250000,433.250000,482.750000
2,2000441674,11570.250000,12723.750000,13764.500000,14633.250000,15625.750000,15808.750000,15839.000000,15732.000000,16327.250000
3,7017966000,3678.610612,3439.005787,2650.188212,2041.683185,1629.577809,1093.070101,964.245914,1178.110637,1343.606838
4,9008948,5746.150000,5572.650000,5342.312500,5225.075000,4915.012500,5099.922500,5263.995000,5773.360000,6042.257500
...,...,...,...,...,...,...,...,...,...,...
200552,18725,2261.500000,391.000000,376.250000,376.750000,377.500000,1861.250000,2355.250000,9751.750000,17992.750000
200553,18734,15894.500000,7698.750000,491.250000,488.500000,519.500000,3787.000000,3936.500000,14884.500000,26570.750000
200554,17668,6770.000000,9113.000000,11086.000000,12423.000000,11535.000000,10784.000000,12153.000000,13167.000000,12566.000000
200555,17205,1577.250000,1689.000000,1824.500000,1779.500000,1521.000000,1317.750000,1334.000000,1611.000000,1622.500000


In [28]:
df = df.merge(df_ene_full, on='COD_ID', how='inner')

In [29]:
df.loc[(df['consumo_minimo_ultimo_ano'] > df['consumo_minimo_ultimo_ano'].describe(percentiles=[0.99])['99%'])].reset_index(drop=True)

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,variacao_percentual_consumo,ENE_04_MA,ENE_05_MA,ENE_06_MA,ENE_07_MA,ENE_08_MA,ENE_09_MA,ENE_10_MA,ENE_11_MA,ENE_12_MA
0,6.128082e+12,VILA LINHA GLORIA S/N,2.0,89890000,5211799,1066000.0,S/N,Ativa,AGROPLANTA PRODUTOS AGROPECUARIOS,SC,...,-4.669048,9.318058e+05,9.165285e+05,8.996255e+05,9.007872e+05,8.663305e+05,9.044310e+05,9.306160e+05,9.431442e+05,9.865020e+05
1,4.581737e+13,RUA AUGUSTO CORREA 214,1.0,66075110,7420001,8599699.0,214,Ativa,NaN,PA,...,-24.326280,1.185869e+06,1.253855e+06,1.325779e+06,1.302967e+06,1.333513e+06,1.355969e+06,1.378955e+06,1.418607e+06,1.359835e+06
2,3.268291e+13,AVENIDA PORTUGAL 1148,1.0,74150030,8630502,8610102.0,1148,Ativa,ELOS CLINICA - ESPACO DE LONGEVIDADE SAUDAVEL,GO,...,6.473055,7.738185e+05,7.795961e+05,7.810334e+05,7.784101e+05,7.437671e+05,7.268087e+05,7.245046e+05,7.423162e+05,7.635042e+05
3,4.454686e+13,AVENIDA BRASIL S/N,1.0,68523000,724302,710301.0,S/N,Ativa,EMASIL MINERACAO,PA,...,-2.773610,8.385473e+05,8.681316e+05,9.212574e+05,9.256954e+05,8.855706e+05,8.410750e+05,8.201967e+05,7.796672e+05,7.808572e+05
4,8.146691e+12,AVENIDA ISABEL DOMINGUES 44,1.0,22763627,1122403,4729699.0,44,Ativa,SUCO SELETO,RJ,...,8.852543,6.710455e+05,6.885622e+05,6.607261e+05,6.522184e+05,6.221694e+05,6.099203e+05,6.141410e+05,6.331859e+05,6.454188e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,NaN,NaN,NaN,14.175-300,2822-4/01,NaN,NaN,NaN,NaN,NaN,...,12.366979,1.228840e+06,1.348562e+06,1.323027e+06,1.268352e+06,1.303238e+06,1.265476e+06,1.286489e+06,1.303736e+06,1.335712e+06
2002,NaN,NaN,NaN,06.455-010,8640-2/01,NaN,NaN,NaN,NaN,NaN,...,4.386949,6.849490e+05,6.833453e+05,6.681267e+05,6.625970e+05,6.368979e+05,6.419677e+05,6.456926e+05,6.658633e+05,6.837569e+05
2003,NaN,NaN,NaN,19.470-000,1099-6/01,NaN,NaN,NaN,NaN,NaN,...,8.025417,2.487801e+06,2.451216e+06,2.503718e+06,2.436404e+06,2.428208e+06,2.450434e+06,2.458540e+06,2.478755e+06,2.530802e+06
2004,NaN,NaN,NaN,06.455-020,1732-0/00,NaN,NaN,NaN,NaN,NaN,...,3.365648,1.564264e+06,1.564235e+06,1.496952e+06,1.455278e+06,1.306064e+06,1.256375e+06,1.238900e+06,1.223579e+06,1.278879e+06


In [30]:
df = df.loc[(df['consumo_minimo_ultimo_ano'] > df['consumo_minimo_ultimo_ano'].describe(percentiles=[0.99])['99%'])].reset_index(drop=True)

In [31]:
df

,cnpj,endereco,matriz_filial,CEP,CNAE,cnae_2_split,numero,situacao_cadastral,nome_fantasia,UF,...,variacao_percentual_consumo,ENE_04_MA,ENE_05_MA,ENE_06_MA,ENE_07_MA,ENE_08_MA,ENE_09_MA,ENE_10_MA,ENE_11_MA,ENE_12_MA
0,6.128082e+12,VILA LINHA GLORIA S/N,2.0,89890000,5211799,1066000.0,S/N,Ativa,AGROPLANTA PRODUTOS AGROPECUARIOS,SC,...,-4.669048,9.318058e+05,9.165285e+05,8.996255e+05,9.007872e+05,8.663305e+05,9.044310e+05,9.306160e+05,9.431442e+05,9.865020e+05
1,4.581737e+13,RUA AUGUSTO CORREA 214,1.0,66075110,7420001,8599699.0,214,Ativa,NaN,PA,...,-24.326280,1.185869e+06,1.253855e+06,1.325779e+06,1.302967e+06,1.333513e+06,1.355969e+06,1.378955e+06,1.418607e+06,1.359835e+06
2,3.268291e+13,AVENIDA PORTUGAL 1148,1.0,74150030,8630502,8610102.0,1148,Ativa,ELOS CLINICA - ESPACO DE LONGEVIDADE SAUDAVEL,GO,...,6.473055,7.738185e+05,7.795961e+05,7.810334e+05,7.784101e+05,7.437671e+05,7.268087e+05,7.245046e+05,7.423162e+05,7.635042e+05
3,4.454686e+13,AVENIDA BRASIL S/N,1.0,68523000,724302,710301.0,S/N,Ativa,EMASIL MINERACAO,PA,...,-2.773610,8.385473e+05,8.681316e+05,9.212574e+05,9.256954e+05,8.855706e+05,8.410750e+05,8.201967e+05,7.796672e+05,7.808572e+05
4,8.146691e+12,AVENIDA ISABEL DOMINGUES 44,1.0,22763627,1122403,4729699.0,44,Ativa,SUCO SELETO,RJ,...,8.852543,6.710455e+05,6.885622e+05,6.607261e+05,6.522184e+05,6.221694e+05,6.099203e+05,6.141410e+05,6.331859e+05,6.454188e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001,NaN,NaN,NaN,14.175-300,2822-4/01,NaN,NaN,NaN,NaN,NaN,...,12.366979,1.228840e+06,1.348562e+06,1.323027e+06,1.268352e+06,1.303238e+06,1.265476e+06,1.286489e+06,1.303736e+06,1.335712e+06
2002,NaN,NaN,NaN,06.455-010,8640-2/01,NaN,NaN,NaN,NaN,NaN,...,4.386949,6.849490e+05,6.833453e+05,6.681267e+05,6.625970e+05,6.368979e+05,6.419677e+05,6.456926e+05,6.658633e+05,6.837569e+05
2003,NaN,NaN,NaN,19.470-000,1099-6/01,NaN,NaN,NaN,NaN,NaN,...,8.025417,2.487801e+06,2.451216e+06,2.503718e+06,2.436404e+06,2.428208e+06,2.450434e+06,2.458540e+06,2.478755e+06,2.530802e+06
2004,NaN,NaN,NaN,06.455-020,1732-0/00,NaN,NaN,NaN,NaN,NaN,...,3.365648,1.564264e+06,1.564235e+06,1.496952e+06,1.455278e+06,1.306064e+06,1.256375e+06,1.238900e+06,1.223579e+06,1.278879e+06


In [32]:
trends = df[ene_mov_cols].values.tolist()

In [33]:

#lm = LinearRegression()
def get_intercept_coef(series):
    lm = SGDRegressor()
    lm.fit(np.array(list(range(0, 9))).reshape(-1, 1), series)
    return lm.intercept_, lm.coef_

intercept_list = []
coef_list = []
std_scaler = StandardScaler()
for serie_list in trends:
    #applying StandartScaler over series_list
    serie_list = std_scaler.fit_transform(np.array(serie_list).reshape(-1, 1))
    intercept, coef = get_intercept_coef(serie_list)
    intercept_list.append(intercept)
    coef_list.append(coef[0])
df_trends = pd.DataFrame({'intercept': intercept_list, 'coef': coef_list})

df_trends.sort_values(by='coef', ascending=False)

scaler = MinMaxScaler(feature_range=(0, 100))
df_trends['coef_scaled'] = scaler.fit_transform(df_trends[['coef']])
df_trends['intercept_scaled'] = scaler.fit_transform(df_trends[['intercept']])


C:\Users\digom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\digom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\digom\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

In [34]:
df_trends['coef'] 

0       0.095763
1       0.228170
2      -0.108595
3      -0.133279
4      -0.133599
          ...   
2001    0.049812
2002   -0.039077
2003    0.039314
2004   -0.217315
2005    0.164856
Name: coef, Length: 2006, dtype: float64

In [35]:
df_final = df.copy()
df_final['trend_coef'] = df_trends['coef']
df_final['trend_coef_scaled'] = df_trends['coef_scaled']
df_final['trend_intercept'] = df_trends['intercept']
df_final['trend_intercept_scaled'] = df_trends['intercept_scaled']
df_final['consumo_medio_ultimo_ano_scaled'] = scaler.fit_transform(df_final[['consumo_medio_ultimo_ano']])
coef_weight = 2
consumo_medio_weight = 1.5
df_final['coef_consumo_medio_mean'] = (coef_weight * df_final['trend_coef_scaled'] + consumo_medio_weight * df_final['consumo_medio_ultimo_ano_scaled'])/(coef_weight + consumo_medio_weight)
df_final['coef_consumo_medio_mean'] = scaler.fit_transform(df_final[['coef_consumo_medio_mean']])
df_final['ene_series'] = df[ene_cols_full].values.tolist()
df_final['mov_avg_series'] = df[ene_mov_cols].values.tolist()
df_final['mov_avg_max'] = df_final['mov_avg_series'].apply(lambda x: max(x))
df_final['ene_series_max'] = df_final['ene_series'].apply(lambda x: max(x))

In [36]:
df_final['LIV'].value_counts()

1    1686
0     320
Name: LIV, dtype: int64

In [37]:
final_cols = ['cnpj', 'endereco', 'matriz_filial', 'CEP', 'CNAE', 'coef_consumo_medio_mean', 'trend_intercept', 'consumo_medio_ultimo_ano_scaled',
              'consumo_medio_ultimo_ano', 'desvio_padrao_consumo_ultimo_ano', 'variacao_percentual_consumo',
              'consumo_minimo_ultimo_ano', 'consumo_maximo_ultimo_ano', 'consumo_final_menos_inicial',
              'nome_municipio', 'UF',  'trend_coef', 'trend_coef_scaled',  'mov_avg_max', 'ene_series_max',
               'ene_series', 'mov_avg_series', 'nome_fantasia', 'LIV', 'POINT_X', 'POINT_Y']
df_final = df_final[final_cols]
df_final.sort_values(by=['LIV', 'coef_consumo_medio_mean'], ascending=[True, False], inplace=True)
df_final.to_csv('./data/energia_com_cnpj_trends.csv', index=False)
df_final[['consumo_medio_ultimo_ano', 'desvio_padrao_consumo_ultimo_ano', 'trend_coef_scaled',
                       'consumo_minimo_ultimo_ano', 'consumo_maximo_ultimo_ano', 'trend_coef','variacao_percentual_consumo']]

,consumo_medio_ultimo_ano,desvio_padrao_consumo_ultimo_ano,trend_coef_scaled,consumo_minimo_ultimo_ano,consumo_maximo_ultimo_ano,trend_coef,variacao_percentual_consumo
1987,6.420210e+06,576297.241077,82.696572,5375605.32,7247225.00,0.177793,3.391083
187,1.138846e+06,85982.692529,98.175509,1009916.00,1242648.96,0.265071,8.875145
1359,1.275154e+06,94569.220889,97.114226,1175487.60,1437349.56,0.259087,-0.262418
1319,2.676880e+06,112816.814450,88.903413,2383700.00,2785385.00,0.212790,3.137455
1250,1.059870e+06,113249.417565,95.134723,896243.90,1207001.12,0.247925,1.487559
...,...,...,...,...,...,...,...
1731,7.441847e+05,72396.952868,4.638260,607682.00,858973.00,-0.262339,-14.133773
654,7.283780e+05,55696.543907,4.316103,635128.50,830515.50,-0.264156,-7.468774
1274,1.260996e+06,185601.542602,2.227201,840280.20,1554088.30,-0.275934,-28.944467
1552,8.401350e+05,95148.162672,2.421288,577371.60,959725.44,-0.274840,-28.398143


In [38]:
df_final.loc[df_final['LIV'] == 0, 'cnpj'].value_counts(dropna=False)

NaN             265
4.581737e+13      1
7.985057e+13      1
4.377652e+13      1
3.637355e+13      1
8.644917e+13      1
3.268291e+13      1
1.053237e+13      1
8.653335e+13      1
1.983460e+13      1
8.630112e+13      1
4.454686e+13      1
8.146691e+12      1
9.750584e+13      1
9.750584e+13      1
4.660969e+13      1
1.879994e+13      1
2.464981e+13      1
4.172213e+12      1
3.014550e+13      1
8.509003e+13      1
4.022859e+13      1
8.331044e+13      1
2.710117e+13      1
2.800057e+12      1
1.894155e+12      1
3.663365e+12      1
3.402832e+13      1
2.815136e+13      1
1.733812e+13      1
7.350344e+12      1
7.556815e+13      1
5.385911e+13      1
3.759576e+13      1
2.016440e+12      1
8.298208e+13      1
7.456210e+12      1
1.068119e+13      1
1.257995e+12      1
8.977416e+13      1
3.195577e+12      1
9.095205e+13      1
8.643304e+13      1
4.667099e+13      1
7.662543e+12      1
6.128082e+12      1
6.243019e+13      1
8.305444e+13      1
6.153538e+13      1
4.461836e+12      1
